In [7]:
import numpy as np
import scipy.io 
import wfdb
import os
import csv

from joblib import Parallel, delayed, parallel_backend
from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [8]:
# Importing the custom denoising functions that I wrote
from utils.denoising_functions import Nvar_Calculation_from_std_dev, NLM_1dDarbon, NLM_1dDarbon_2D_full
from utils.denoising_functions import Butterworth_lowpass_filter, remove_baseline_loess
from utils.PlottingFunctions import quick_plot, plot_ecg_stacked
from utils.ExtractingGroups import extract_group

In [9]:
#Importing the tensorflow package and layers

import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, ReLU, Add, Input
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
# Original WFDB folder
base_path = "/home/lewisa13/ECG_Data/physionet.org/files/ecg-arrhythmia/1.0.0/WFDBRecords"
output_path = "CleanedECG" # Folder to save denoised signal

os.makedirs(output_path, exist_ok = True)

fs = 500 # Sampling frequency is 500 Hz
cutoff = 50 #Setting a cutoff of 50 Hz for low pass filtering

## NLM Denoising Parameters
# PatchHW is the patch half width
#Patch length = 2* PatchHW +1
#At 500 Hz sampling, 1 sample = 2ms, the QRS complex is about 40-60 samples
# Want the patch length to be less than QRS comples, so between 10-20 samples
# PatchLength used in the paper was 10, So recommened PatchHW is 5
PatchHW = 5

# P is the search window, controls how far the algorithm searches for similar patches
P = 25



In [5]:
# --------------------------
# Function to process one file
# --------------------------
def process_ecg_file(mat_path, fs, cutoff, P, PatchHW):
    record_id = os.path.splitext(os.path.basename(mat_path))[0]
    output_file = os.path.join(save_folder, f"{record_id}.npy")

    # Skip if already processed
    if os.path.exists(output_file):
        print("Skipping:", record_id)
        return None

    print("Processing:", record_id)

    header_path = mat_path.replace(".mat", ".hea")

    # --------------------------
    # Load ECG
    # --------------------------
    mat_data = scipy.io.loadmat(mat_path)
    ecg = mat_data['val']  # shape (n_leads, n_samples)

    # --------------------------
    # Butterworth low-pass filter
    # --------------------------
    ecg_low_pass = Butterworth_lowpass_filter(ecg, fs, cutoff, order=2)

    # --------------------------
    # LOESS baseline removal (threading-safe)
    # --------------------------
    n_leads = ecg_low_pass.shape[0]

    
    ecg_loess = np.array([
        remove_baseline_loess(ecg_low_pass[ilead, :], fs, frac=0.05, it=3)
        for ilead in range(n_leads)
    ])


    # --------------------------
    # Compute noise variance for NLM
    # --------------------------
    Nvar = Nvar_Calculation_from_std_dev(ecg_loess)

    # --------------------------
    # NLM denoising (non-vectorized per lead)
    # --------------------------
    ecg_denoised = np.array([
        NLM_1dDarbon(ecg_loess[ilead, :], Nvar, P, PatchHW)
        for ilead in range(n_leads)
    ])

    # plot_ecg_stacked(ecg_denoised, fs=500, n_signals=5, lead_names=None,spacing_factor=2, linewidth=1)
    # fig = plt.gcf()
    # fig.savefig(output_plot_path)
    # plt.close(fig)
    
    # --------------------------
    # Save denoised ECG
    # --------------------------
    os.makedirs(save_folder, exist_ok=True)
    
    np.save(os.path.join(save_folder, f"{record_id}.npy"), ecg_denoised)
    print("Saved:", record_id)
    
    # --------------------------
    # Extract grouedp labels
    # --------------------------
    group_code = extract_group(header_path)
    label_row = [record_id, "|".join(group_code)]

    return label_row

# --------------------------
# Main function to process all .mat files
# --------------------------
def process_all_ecg(base_path, fs, cutoff, P, PatchHW):
    # Gather all .mat files recursively
    print(base_path)
    all_mat_files = [
        os.path.join(root, f)
        for root, dirs, files in os.walk(base_path)
        for f in files
        if f.endswith(".mat")
    ]

    print(f"Found {len(all_mat_files)} .mat files")

    #Selecting a few test files for now. Working on a code to process the files batchwise
    test_files = all_mat_files[110:120]
    
    with parallel_backend("loky",n_jobs=16):
        label_rows = Parallel(verbose=10)(
            delayed(process_ecg_file)(mat_path, fs, cutoff, P, PatchHW)
            for mat_path in test_files
    )

    # Process files in parallel (threading backend for debugger-safe)
    # with parallel_backend('threading', n_jobs=-1):
    #     label_rows = Parallel()(
    #         delayed(process_ecg_file)(mat_path, fs, cutoff, P, PatchHW)
    #         for mat_path in all_mat_files
    #     )

    # label_rows = [
    #     process_ecg_file(mat_path, fs, cutoff, P, PatchHW)
    #     for mat_path in all_mat_files[100:105]
    # ]
    
    return label_rows



# --------------------------
# Usage
# --------------------------
if __name__ == "__main__":
    base_path =  "/home/lewisa13/ECG_Data/physionet.org/files/ecg-arrhythmia/1.0.0/WFDBRecords"
    save_folder = "/home/lewisa13/CleanedECG"
    output_plot_path = "/home/lewisa13/CleanedECGPlots"
    fs = 500          # sampling frequency
    cutoff = 40       # low-pass cutoff
    P = 20            # NLM search window
    PatchHW = 3       # NLM patch half-width

    import time

    start = time.time()

    labels = process_all_ecg(base_path, fs, cutoff, P, PatchHW)

    end = time.time()

    runtime = end - start
    print(f"Total runtime: {runtime:.1f} s ({runtime/60:.1f} min)")
    %print("All files processed. Label rows:", labels)


/home/lewisa13/ECG_Data/physionet.org/files/ecg-arrhythmia/1.0.0/WFDBRecords
Found 45152 .mat files


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   3 out of  10 | elapsed:    8.9s remaining:   20.8s
[Parallel(n_jobs=16)]: Done   5 out of  10 | elapsed:    9.0s remaining:    9.0s
[Parallel(n_jobs=16)]: Done   7 out of  10 | elapsed:    9.1s remaining:    3.9s


Total runtime: 9.7 s (0.2 min)


[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:    9.6s finished
UsageError: Line magic function `%print("All` not found.


In [12]:
# Open a new csv file to write the group label for each ECG
# If the file exists, it overwrites it, if ti doesn't exist it creates a new file
# newline= "" ensures that python doesn't insert extra blank lines when writing row on windows
# open automatically closes the file when done
with open("labels.csv","w",newline = "") as f:
    #Create a write object that lets you writerows to the CSV file easily
    writer = csv.writer(f)
    #.writerow wrtes a single row to the CSV file. Here it is the header row
    writer.writerow(["record_id","group_code"])
    # .writerows writes multiple rows at once
    writer.writerows(labels)

Processing: JS00117
Saved: JS00117
Processing: JS00115
Saved: JS00115
Processing: JS00121
Saved: JS00121
Processing: JS00118
Saved: JS00118
Processing: JS00124
Saved: JS00124
Processing: JS00122
Saved: JS00122
Processing: JS00123
Saved: JS00123
Processing: JS00120
Saved: JS00120
Processing: JS00119
Saved: JS00119
Processing: JS00116
Saved: JS00116


In [16]:
df = pd.read_csv("labels.csv")

le = LabelEncoder() # LabelEncoder is a tool from scikit-learn used to convert categorical labesl 
y = le.fit_transform(df["group_code"])

print(le.classes_)

['A|F|I|B' 'S|B']


In [ ]:
# Defining the intial block

def initialConvBlock(input):
    # Defines the inital block of the neural network
    #First block has 64 filters, filter length of 16 and stride of 4, "same' is added to determine the required padding
    x1 = Conv1D(filters = 64, kernel_size = 16,padding = 'same',strides = 4)(input)
    x1 = BatchNormalization()(x1)
    x1 = ReLU()(x1)

    return x1

In [ ]:
# Defining the resblk block using a function, N_filters is used as an input since the filters double ever two res blocks

def resblk(input, N_filters, stride_size, dropout_rate):
    # resblk has one input which splits into two branches coming from the previous layer

    #Max pooling branch is the first branch
    x1 = MaxPooling1D(pool_size = 4, strides = stride_size, padding = "valid")(input)
    x1 = Conv1D(filters = N_filters, kernel_size = 1, padding = "valid")(x1)

    # Conv layer is the second branch 
    x2 = Conv1D(filters = N_filters, kernel_size = 16, strides = stride_size, padding = "same")(input)
    x2 = BatchNormalization()(x2)
    x2 = ReLU()(x2)
    x2 = Dropout(dropout_rate)(x2)

    x2 = Conv1D(filters = N_filters, kernel_size = 16, strides = 1, padding = "same")(x2)

    # Add the output from the two branches
    x3 = Add()([x1, x2])

    # Apply the batch normalization, ReLU and Dropout layers
    x3 = BatchNormalization()(x3)
    x3 = ReLU()(x3)
    x3 = Dropout(dropout_rate)(x3)
    
    return x3
    

In [ ]:
# Putting together the entire network
num_classes = 4

inputs = Input(shape = (4096,12))

convblock1 = initialConvBlock(inputs)

resblk1 = resblk(convblock1, N_filters = 64, stride_size = 4, dropout_rate = 0.5)

resblk2 = resblk(resblk1, N_filters = 64, stride_size = 4, dropout_rate = 0.5)

resblk3 = resblk(resblk2, N_filters = 128, stride_size = 4, dropout_rate = 0.5)

resblk4 = resblk(resblk3, N_filters = 192, stride_size = 4, dropout_rate = 0.5)

flatten_layer = Flatten()(resblk4)

Output_dense_layer = Dense(units = num_classes, activation = 'sigmoid')(flatten_layer)

model = Model(inputs= inputs, outputs = Output_dense_layer)

model.summary()

# Beta=1.0 makes it a standard F1 score
# average='micro' aggregates TPs, FPs, and FNs across all classes
micro_f1 = tf.keras.metrics.FBetaScore(beta=1.0, average='micro', threshold=0.5)

# precision = tf.keras.Precision()
# recall = tf.keras.Recall()
optimizer = Adam(learning_rate = 0.001)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy', micro_f1])
